## Objectifs
* align geo mapping and extent s1 / s2
* convert s2 reflectance


In [36]:
import geopandas as gpd 
import numpy as np 
import os
import matplotlib.pyplot as plt 
import rasterio as rio 
import rioxarray as rioxr
import glob 
import xarray as xr
import folium
from shapely.geometry import box

In [2]:
EPSG = 32622

In [3]:
project_dir = "/home/rustt/Documents/Projects/S1_S2_classification"
out_dir = os.path.join(project_dir, "sentinel_classification/data/processed")

In [4]:
labels_path = os.path.join(project_dir, "sentinel_classification/data/processed/ROI_Classif_corrected.shp")
study_area = os.path.join(project_dir, "sentinel_classification/data/raw/area/StudyArea.shp")
s1_dir_path = os.path.join(project_dir, "Data/Output/Spk/p47")
s1_folder = os.path.join(s1_dir_path, "S1B_IW_GRDH_1SDV_20180114T214240_20180114T214308_009173_0106C1_7F29")
s2_stack_pan_path = os.path.join(project_dir, "Data/Output/S2_10_20_stack_pansharp.tif")



In [5]:
area = gpd.read_file(study_area).to_crs(EPSG)

labels = gpd.read_file(labels_path)

s2_stack = rioxr.open_rasterio(s2_stack_pan_path)

#s1_raster = rioxr.open_rasterio(os.path.join(s1_folder, os.listdir(s1_folder)[0]))

### Rasters specs

In [6]:
def print_raster(raster):
    print(
        f"shape: {raster.rio.shape}\n"
        f"resolution: {raster.rio.resolution()}\n"
        f"bounds: {raster.rio.bounds()}\n"
        f"count: {raster.count().item()}\n"
        f"CRS: {raster.rio.crs}\n"
    )

In [7]:
print("S2 raster")
print(print_raster(s2_stack))

S2 raster
shape: (10980, 10980)
resolution: (10.0, -10.0)
bounds: (300000.0, 490200.0, 409800.0, 600000.0)
count: 1205604000
CRS: EPSG:32622

None


In [8]:
print("S1 raster")
print(print_raster(s1_raster))

S1 raster
shape: (4056, 5610)
resolution: (10.0, -10.0)
bounds: (-5864533.0, 534178.0, -5808433.0, 574738.0)
count: 22754160
CRS: EPSG:3857

None


In [10]:
def create_dir(path, name_dir):
    sub_out_dir = os.path.join(path, name_dir)
    if not os.path.exists(sub_out_dir):
        os.mkdir(sub_out_dir)
    return sub_out_dir

### Align rasters

#### Reproject S1

In [12]:
sub_out_dir = create_dir(out_dir, "s1")

Missing pixels = bbox larger than real raster => we restrict the bbox and clip s1

In [27]:
MARGIN = 200
bounds = area.total_bounds
bounds = (
   np.round(bounds[0]+MARGIN),
   np.round(bounds[1]+MARGIN), 
   np.round(bounds[2]-MARGIN), 
   np.round(bounds[3]-MARGIN), 

)

area_geom = [_ for _ in area.geometry]

tif_list = glob.glob(s1_folder+"/*.tif")
# reproject every polarization tif
for i, file_path in enumerate(tif_list):
    file_name = file_path.split("/")[-1]

    # ensure to have same clipping
    s1_reproject = (
         rioxr.open_rasterio(file_path)
         .rio.reproject(EPSG)          
         .rio.clip_box(*bounds)
    )

    s1_reproject.rio.to_raster(os.path.join(sub_out_dir, f"clip_{file_name}"))
    print(f"{i+1}/{len(tif_list)} : raster {file_name} reprojected ! ")

1/3 : raster S1B_20180114_VV_Sig0_Ortho_TempFilt_W11_dB.tif reprojected ! 
2/3 : raster S1B_20180114__Sig0_Ortho_TempFilt_W11_VHdB-VVdB.tif reprojected ! 
3/3 : raster S1B_20180114_VH_Sig0_Ortho_TempFilt_W11_dB.tif reprojected ! 


In [15]:
# how to do it fast with xarray ?
s1_arr = s1_reproject.data
assert np.count_nonzero(np.where(s1_arr.flatten() == 0)) == 0
del s1_arr

#### Stack s1 polarization

In [ ]:
polarization_code = ["_VV_", "_VH_", "VHdB-VVdB"]

array_stack = {}

for file_name in os.listdir(sub_out_dir): 
    file_path = os.path.join(sub_out_dir, file_name)
    da = rioxr.open_rasterio(file_path)
    pola = [code.replace("_", "").replace("dB", "").strip() for code in polarization_code if code in file_name] # TODO catch KeyError
    if pola:
        array_stack[f"{pola[0]}"] = (("y", "x"), da.sel(band=1).values)

# S1 product already calibrated    
coords = dict(y=da.coords["y"].values, x=da.coords["x"].values)
attrs = da.attrs 

s1_stack = xr.Dataset(
    data_vars=array_stack, 
    coords=coords, 
    attrs=attrs 
).rio.write_crs(EPSG)

s1_stack.rio.to_raster(os.path.join(out_dir, "s1", "clip_s1_stack_polarization.tif"))

clip_S1B_20180114_VV_Sig0_Ortho_TempFilt_W11_dB.tif
clip_s1_stack_polarization.tif
clip_S1B_20180114__Sig0_Ortho_TempFilt_W11_VHdB-VVdB.tif
clip_S1B_20180114_VH_Sig0_Ortho_TempFilt_W11_dB.tif


In [51]:
s1_stack.rio.crs

CRS.from_epsg(32622)

#### Clip s2

On align s2 sur s1

In [16]:
s1_reproject = rioxr.open_rasterio(os.path.join(sub_out_dir, "clip_S1B_20180114_VV_Sig0_Ortho_TempFilt_W11_dB.tif"))
sub_out_dir = create_dir(out_dir, "s2")

#### S2 as xr.Dataset
* Convert dimension "band" as variables

In [2]:
# flat dim "bands" as variables
s2_clip_match = s2_stack.rio.reproject_match(s1_reproject)

NameError: name 's2_stack' is not defined

In [17]:
s2_clip_match = s2_stack.rio.reproject_match(s1_reproject)
s2_clip_match.rio.to_raster(os.path.join(sub_out_dir, "clip_s2_pan_sharp.tif"))

In [ ]:
print_raster(s1_reproject)

In [18]:
print("S2 Original:\n----------------\n")
print_raster(s2_stack)
print("S2 reproject:\n----------------\n")
print_raster(s2_clip_match)
print("S1 Original:\n----------------\n")
print_raster(s1_raster)
print("S1 reproject:\n----------------\n")
print_raster(s1_reproject)

S2 Original:
----------------

shape: (10980, 10980)
resolution: (10.0, -10.0)
bounds: (300000.0, 490200.0, 409800.0, 600000.0)
CRS: EPSG:32622

S2 reproject:
----------------

shape: (4011, 5592)
resolution: (9.93904234597862, -9.939042345978647)
bounds: (313658.24854230555, 530088.5301961509, 369237.373341018, 569954.0290458713)
CRS: EPSG:32622

S1 Original:
----------------

shape: (4056, 5610)
resolution: (10.0, -10.0)
bounds: (-5864533.0, 534178.0, -5808433.0, 574738.0)
CRS: EPSG:3857

S1 reproject:
----------------

shape: (4011, 5592)
resolution: (9.93904234597862, -9.939042345978647)
bounds: (313658.24854230555, 530088.5301961509, 369237.373341018, 569954.0290458713)
CRS: EPSG:32622

